## DataMining Assignment 2

In [17]:
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [18]:
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')
tags = pd.read_csv('data/tags.csv')
links = pd.read_csv('data/links.csv')

In [19]:
print("Totoal number of Movies: "+str(len(movies)))
print("Totoal number of Users: "+str(ratings.userId.nunique()))


Totoal number of Movies: 9742
Totoal number of Users: 610


Merging data

### Note: From README.txt
_Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970_

In our case of study this attribute has no importance, so we will get rid of it

In [20]:
tags.drop(columns=['timestamp'], inplace=True)
ratings.drop(columns=['timestamp'], inplace=True)

In [21]:
merged_data = pd.merge(ratings, movies, on='movieId')
merged_data = pd.merge(merged_data, tags, on=['userId', 'movieId'], how='left')

merged_data.dropna(inplace=True)

merged_data.drop_duplicates(inplace=True)

merged_data

,userId,movieId,rating,title,genres,tag
241,2,60756,5.0,Step Brothers (2008),Comedy,funny
242,2,60756,5.0,Step Brothers (2008),Comedy,Highly quotable
243,2,60756,5.0,Step Brothers (2008),Comedy,will ferrell
252,2,89774,5.0,Warrior (2011),Drama,Boxing story
253,2,89774,5.0,Warrior (2011),Drama,MMA
...,...,...,...,...,...,...
99967,606,6107,4.0,Night of the Shooting Stars (Notte di San Lore...,Drama|War,World War II
100087,606,7382,4.5,I'm Not Scared (Io non ho paura) (2003),Drama|Mystery|Thriller,for katie
101553,610,3265,5.0,Hard-Boiled (Lat sau san taam) (1992),Action|Crime|Drama|Thriller,gun fu
101554,610,3265,5.0,Hard-Boiled (Lat sau san taam) (1992),Action|Crime|Drama|Thriller,heroic bloodshed


In [22]:
average_rating = merged_data.groupby('movieId')['rating'].mean().reset_index()
average_rating.columns = ['movieId', 'average_rating']

merged_data = pd.merge(merged_data, average_rating, on='movieId')

merged_data

,userId,movieId,rating,title,genres,tag,average_rating
0,2,60756,5.0,Step Brothers (2008),Comedy,funny,4.187500
1,2,60756,5.0,Step Brothers (2008),Comedy,Highly quotable,4.187500
2,2,60756,5.0,Step Brothers (2008),Comedy,will ferrell,4.187500
3,2,89774,5.0,Warrior (2011),Drama,Boxing story,5.000000
4,2,89774,5.0,Warrior (2011),Drama,MMA,5.000000
...,...,...,...,...,...,...,...
3471,606,6107,4.0,Night of the Shooting Stars (Notte di San Lore...,Drama|War,World War II,4.000000
3472,606,7382,4.5,I'm Not Scared (Io non ho paura) (2003),Drama|Mystery|Thriller,for katie,4.166667
3473,610,3265,5.0,Hard-Boiled (Lat sau san taam) (1992),Action|Crime|Drama|Thriller,gun fu,5.000000
3474,610,3265,5.0,Hard-Boiled (Lat sau san taam) (1992),Action|Crime|Drama|Thriller,heroic bloodshed,5.000000


In [23]:
new_merged_data = copy.deepcopy(merged_data)

In [24]:
genres_list = []

for index, row in new_merged_data.iterrows():
    genres = row['genres'].split('|')
    for genre in genres:
        if genre not in genres_list:
            genres_list.append(genre)

for genre in genres_list:
    new_merged_data[genre] = 0

for index, row in new_merged_data.iterrows():
    genres = row['genres'].split('|')
    for genre in genres:
        new_merged_data.at[index, genre] = 1

new_merged_data

,userId,movieId,rating,title,genres,tag,average_rating,Comedy,Drama,Crime,...,Adventure,Children,Fantasy,Animation,Horror,Romance,Musical,Western,Film-Noir,(no genres listed)
0,2,60756,5.0,Step Brothers (2008),Comedy,funny,4.187500,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,60756,5.0,Step Brothers (2008),Comedy,Highly quotable,4.187500,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,60756,5.0,Step Brothers (2008),Comedy,will ferrell,4.187500,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,89774,5.0,Warrior (2011),Drama,Boxing story,5.000000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2,89774,5.0,Warrior (2011),Drama,MMA,5.000000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471,606,6107,4.0,Night of the Shooting Stars (Notte di San Lore...,Drama|War,World War II,4.000000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3472,606,7382,4.5,I'm Not Scared (Io non ho paura) (2003),Drama|Mystery|Thriller,for katie,4.166667,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3473,610,3265,5.0,Hard-Boiled (Lat sau san taam) (1992),Action|Crime|Drama|Thriller,gun fu,5.000000,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3474,610,3265,5.0,Hard-Boiled (Lat sau san taam) (1992),Action|Crime|Drama|Thriller,heroic bloodshed,5.000000,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [25]:
new_merged_data.drop(columns=['genres', 'title', '(no genres listed)'], inplace=True)

In [26]:
tags = []

for index, row in new_merged_data.iterrows():
    if row['tag'] not in tags:
        tags.append(row['tag'])


for i in range(len(tags)):
    new_merged_data.loc[new_merged_data['tag'] == tags[i], 'tag'] = i+1


new_merged_data

,userId,movieId,rating,tag,average_rating,Comedy,Drama,Crime,Thriller,War,...,IMAX,Adventure,Children,Fantasy,Animation,Horror,Romance,Musical,Western,Film-Noir
0,2,60756,5.0,1,4.187500,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,60756,5.0,2,4.187500,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,60756,5.0,3,4.187500,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,89774,5.0,4,5.000000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,89774,5.0,5,5.000000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471,606,6107,4.0,69,4.000000,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3472,606,7382,4.5,1540,4.166667,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3473,610,3265,5.0,1541,5.000000,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3474,610,3265,5.0,1542,5.000000,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


## Normalization and Standarization

### 1. Normalization

By applying Min-max

In [30]:
normalize_data = copy.deepcopy(new_merged_data)
scaler = MinMaxScaler()
columns_to_normalize = ['userId', 'movieId', 'rating', 'average_rating', 'tag', ]
normalize_data[columns_to_normalize] = scaler.fit_transform(normalize_data[columns_to_normalize])
new_merged_data

,userId,movieId,rating,tag,average_rating,Comedy,Drama,Crime,Thriller,War,...,IMAX,Adventure,Children,Fantasy,Animation,Horror,Romance,Musical,Western,Film-Noir
0,2,60756,5.0,1,4.187500,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,60756,5.0,2,4.187500,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,60756,5.0,3,4.187500,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,89774,5.0,4,5.000000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,89774,5.0,5,5.000000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471,606,6107,4.0,69,4.000000,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3472,606,7382,4.5,1540,4.166667,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3473,610,3265,5.0,1541,5.000000,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3474,610,3265,5.0,1542,5.000000,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Standarization

By applying StandardScaler

In [32]:
standarize_data = copy.deepcopy(new_merged_data)
scaler = StandardScaler()
columns_to_standardize = ['userId', 'movieId', 'rating', 'average_rating', 'tag', ]
standarize_data[columns_to_standardize] = scaler.fit_transform(standarize_data[columns_to_standardize])
standarize_data

,userId,movieId,rating,tag,average_rating,Comedy,Drama,Crime,Thriller,War,...,IMAX,Adventure,Children,Fantasy,Animation,Horror,Romance,Musical,Western,Film-Noir
0,-2.648338,0.742016,1.147488,-1.420143,0.209015,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-2.648338,0.742016,1.147488,-1.417779,0.209015,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-2.648338,0.742016,1.147488,-1.415415,0.209015,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-2.648338,1.399547,1.147488,-1.413051,1.204057,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-2.648338,1.399547,1.147488,-1.410687,1.204057,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471,1.090911,-0.496298,-0.019642,-1.259396,-0.020611,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3472,1.090911,-0.467407,0.563923,2.217943,0.183501,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3473,1.115674,-0.560696,1.147488,2.220307,1.204057,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3474,1.115674,-0.560696,1.147488,2.222671,1.204057,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
